In [ ]:
import numpy as np
import multiprocessing as mp
import os
import cv2
from scipy import ndimage
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tnrange
from keras.models import Sequential, Model
from tensorflow.keras.models import load_model

In [ ]:
def lee_alfabetos(ruta_al_directorio_del_alfabeto, nombre_del_alfabeto, max_imagenes=20):
    """
    Lee las primeras 'max_imagenes' imágenes de los alfabetos que contenga el directorio dado.
    """
    datax = []  # Lista para almacenar las imágenes
    datay = []  # Lista para almacenar las etiquetas de las imágenes

    # Lista todas las imágenes disponibles en el directorio del alfabeto
    imagenes = os.listdir(ruta_al_directorio_del_alfabeto)

    # Si no hay suficientes imágenes en el directorio, se retorna un mensaje
    if len(imagenes) < max_imagenes:
        return None, f"{nombre_del_alfabeto} ({len(imagenes)} imágenes disponibles)"

    # Selecciona las primeras 'max_imagenes' imágenes
    imagenes = imagenes[:max_imagenes]

    # Procesa cada imagen individualmente
    for img in imagenes:
        # Carga la imagen
        imagen = cv2.imread(os.path.join(ruta_al_directorio_del_alfabeto, img))

        # Rota la imagen a diferentes ángulos (90, 180, 270 grados)
        rotada_90 = ndimage.rotate(imagen, 90)
        rotada_180 = ndimage.rotate(imagen, 180)
        rotada_270 = ndimage.rotate(imagen, 270)

        # Agrega la imagen original y sus rotaciones a 'datax'
        datax.extend((imagen, rotada_90, rotada_180, rotada_270))

        # Agrega etiquetas para cada imagen en 'datay'
        datay.extend((
            ruta_al_directorio_del_alfabeto + '_' + '_0',
            ruta_al_directorio_del_alfabeto + '_' + '_90',
            ruta_al_directorio_del_alfabeto + '_' + '_180',
            ruta_al_directorio_del_alfabeto + '_' + '_270'
        ))

    # Convierte las listas en arrays de numpy para su manipulación posterior
    return np.array(datax), np.array(datay)

def preproces_Mini_ImageNet(directorio_principal, max_imagenes_por_clase=600):
    """
    Llama a la función lee_alfabetos para leer las primeras 'max_imagenes_per_class' imágenes de cada uno de los alfabetos del directorio principal.
    Si no hay suficientes imágenes en una clase, se incluirá un mensaje en los resultados.
    """
    datax = np.zeros((20, 84, 84, 3))  # Array inicial para almacenar las imágenes
    datay = []  # Lista para almacenar las etiquetas

    # Utiliza múltiples procesos para leer imágenes en paralelo
    pool = mp.Pool(mp.cpu_count())

    # Llama a la función 'lee_alfabetos' para cada directorio en el directorio principal
    resultados = [pool.apply(lee_alfabetos,
                             args=(
                                 os.path.join(directorio_principal, directorio) + '/',
                                 directorio,
                                 max_imagenes_por_clase
                             )) for directorio in os.listdir(directorio_principal)]

    # Cierra el pool de procesos
    pool.close()

    # Procesa los resultados obtenidos
    for resultado in resultados:
        if resultado[0] is not None:
            # Si 'datax' sigue siendo un array inicial (todos ceros), lo reemplaza con el nuevo resultado
            if datax.shape == (20, 84, 84, 3) and np.all(datax == 0):
                datax = resultado[0]
                datay = resultado[1]
            else:
                # Si no, concatena los resultados
                datax = np.vstack([datax, resultado[0]])
                datay = np.concatenate([datay, resultado[1]])
        else:
            # Si no hay suficientes imágenes para un alfabeto, imprime el mensaje
            print(resultado[1])

    return datax, datay


In [ ]:
# Descomprimir los archivos de train, validacion y test
!tar -xf train.tar
!tar -xf val.tar
!tar -xf test.tar

In [ ]:
# Preprocesar las imágenes de Mini-ImageNet y guardarlas en sus respectivas variables
trainx, trainy = preproces_Mini_ImageNet('train')
valx, valy = preproces_Mini_ImageNet('val')
testx, testy = preproces_Mini_ImageNet('test')

# Como no se va a hacer validación se pueden usar estas imágenes para entrenar.
# Aun así en el estudio del trabajo no se realiza este paso. Se pueden comentar estas lineas para no hacerlo.
trainx = np.concatenate((trainx, valx), axis=0)
trainy=np.concatenate((trainy, valy))

In [ ]:
trainx.shape, trainy.shape, testx.shape, testy.shape

((192000, 84, 84, 3), (192000,), (48000, 84, 84, 3), (48000,))

In [ ]:
def crea_muestra(n_way, n_support, n_query, datax, datay):
    """
    Crea una muestra aleatoria de tamaño n_support+n_query, de imágenes de n_way clases.

    Entrada:
        n_way (int): número de clases
        n_support (int): número de imágenes en el conjunto soporte por clase
        n_query (int): número de imágenes para clasificar por clase
        datax (np.array): conjunto de imágenes
        datay (np.array): conjunto de etiquetas
    Salida:
        (dict) de:
            (tf.Tensor): Muestra de imágenes de tamaño (n_way, n_support+n_query, (dim))
            (int): n_way
            (int): n_support
            (int): n_query
    """

    # Obtiene las clases únicas presentes en las etiquetas
    clases_unicas = np.unique(datay)

    # Elige aleatoriamente 'n_way' clases únicas
    clases_elegidas = np.random.choice(clases_unicas, n_way, replace=False)

    # Suma el número de imágenes de soporte y consulta por clase
    ejemplos_por_clase = n_support + n_query

    # Crea un array con índices de todas las etiquetas
    indices = np.arange(len(datay))

    muestra = []

    # Para cada clase elegida:
    for clase in clases_elegidas:
        # Obtiene los índices donde la clase actual está presente en las etiquetas
        indices_clase = indices[datay == clase]

        # Elige aleatoriamente 'ejemplos_por_clase' índices de esa clase
        indices_seleccionados = np.random.choice(indices_clase, ejemplos_por_clase, replace=False)

        # Agrega los ejemplos seleccionados a la muestra
        muestra.append(datax[indices_seleccionados])

    # Convierte la lista de muestras en un tensor de TensorFlow
    muestra = tf.convert_to_tensor(np.array(muestra), dtype=tf.float32)

    # Retorna un diccionario con la muestra, n_way, n_support y n_query
    return {
        'imagenes': muestra,
        'n_way': n_way,
        'n_support': n_support,
        'n_query': n_query
    }


In [ ]:
def red_convolucional(tamaño_entrada, dimension_caracteristicas):
    """
    Construye y devuelve un modelo de red neuronal convolucional con el tamaño de entrada y
    dimensión de características especificados.

    Entrada:
        tamaño_entrada (tuple): Dimensiones de entrada de la imagen, por ejemplo: (alto, ancho, canales).
        dimension_caracteristicas (int): Número de nodos en la última capa densa.

    Salida:
        (tf.keras.Sequential): Modelo de red convolucional.
    """

    # Inicializa un modelo secuencial
    model = tf.keras.Sequential()

    # Agrega la primera capa convolucional
    model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=tamaño_entrada))  # Convolución con 64 filtros de tamaño 3x3
    model.add(layers.BatchNormalization())  # Normalización por lotes
    model.add(layers.ReLU())  # Función de activación ReLU
    model.add(layers.MaxPooling2D((2, 2)))  # Pooling para reducir dimensiones

    # Agrega 3 bloques de capas convolucionales más, sin indicar para estos el input_shape
    for _ in range(3):
        model.add(layers.Conv2D(64, (3, 3), padding='same'))  # Convolución con 64 filtros de tamaño 3x3
        model.add(layers.BatchNormalization())  # Normalización por lotes
        model.add(layers.ReLU())  # Función de activación ReLU
        model.add(layers.MaxPooling2D((2, 2)))  # Pooling para reducir dimensiones

    # Convierte la salida de la última capa de pooling a un vector 1D
    model.add(layers.Flatten())

    # Agrega una capa densa con "dimension_caracteristicas" nodos
    model.add(layers.Dense(dimension_caracteristicas))

    return model


In [ ]:
def euclidean_dist(x, y):
    """
    Calcula la distancia euclidea al cuadrado entre x e y.

    Entrada:
        x (tf.Tensor): tamaño (n, d). n es n_way*n_query.
        y (tf.Tensor): tamaño (m, d). m es n_way.

    Salida:
        tf.Tensor: tamaño (n, m). Para cada imagen de consulta, las distancias a cada uno de los prototipos.
    """

    # Obtener las dimensiones de los tensores x e y
    n = tf.shape(x)[0]  # Número de filas de x
    m = tf.shape(y)[0]  # Número de filas de y
    d = tf.shape(x)[1]  # Número de columnas de x, que debería ser igual al número de columnas de y

    # Expande las dimensiones de x para que se pueda difundir (broadcast) contra y
    x = tf.expand_dims(x, 1)   # Añade una nueva dimensión en el índice 1
    x = tf.tile(x, (1, m, 1))  # Repite x m veces a lo largo de la nueva dimensión creada

    # Expande las dimensiones de y para que se pueda difundir (broadcast) contra x
    y = tf.expand_dims(y, 0)   # Añade una nueva dimensión en el índice 0
    y = tf.tile(y, (n, 1, 1))  # Repite y n veces a lo largo de la nueva dimensión creada

    # Calcula la distancia euclidiana al cuadrado entre x e y
    # Reduce a lo largo del eje 2 (dimensión d) después de calcular el cuadrado de las diferencias
    return tf.reduce_sum(tf.square(x - y), axis=2)


In [ ]:
def floss(y_true, y_pred):
    """
    Calcula la función de pérdida personalizada para dos vectores: uno que contiene las etiquetas verdaderas y
    otro que contiene las probabilidades predichas de las clases.

    Entrada:
        y_true (tf.Tensor): Tensor que contiene las etiquetas verdaderas en formato one-hot encoding.
        y_pred (tf.Tensor): Tensor que contiene las probabilidades predichas de las clases.

    Salida:
        tf.Tensor: Un valor escalar que representa la pérdida calculada.
    """

    # Multiplica elemento a elemento las etiquetas verdaderas con las probabilidades predichas
    # Luego, sumamos esos valores a lo largo del último eje (axis=-1)
    # Finalmente, calculamos el negativo de la media de esos valores sumados

    loss_val = -tf.reduce_mean(tf.reduce_sum(y_true * y_pred, axis=-1))

    return loss_val


In [ ]:
def accuracyProt(y_true, y_pred, n_way, n_query):
    """
    Calcula la precisión de las predicciones basándose en etiquetas verdaderas y predicciones.

    Entrada:
        y_true (tf.Tensor): Tensor que contiene las etiquetas verdaderas en formato one-hot encoding.
        y_pred (tf.Tensor): Tensor que contiene las probabilidades predichas de las clases.
        n_way (int): Número de clases únicas.
        n_query (int): Número de imágenes para clasificar por clase.

    Salida:
        float: Precisión de las predicciones.
    """

    # Encuentra el índice del valor máximo en y_pred a lo largo del eje 1
    # Esto nos dará la clase predicha para cada muestra.
    predicciones = tf.argmax(y_pred, axis=1)

    # Encuentra el índice del valor máximo en y_true a lo largo del eje 1
    # Esto nos dará la etiqueta verdadera para cada muestra.
    etiquetas_correctas = tf.argmax(y_true, axis=1)

    # Compara las predicciones con las etiquetas correctas y cuenta cuántas coinciden.
    total_correct = tf.reduce_sum(tf.cast(tf.equal(predicciones, etiquetas_correctas), tf.int32))

    # Calcula la precisión dividiendo el número total de predicciones correctas
    # por el número total de predicciones (n_way * n_query).
    accuracyp = total_correct / (n_way * n_query)

    return accuracyp


In [ ]:
def Proto(sample, n_way, n_support, n_query, model):
    """
    Implementa el Prototipo de redes para el aprendizaje de pocas tomas.

    Entrada:
        sample (dict): Diccionario que contiene las imágenes.
        n_way (int): Número de clases únicas.
        n_support (int): Número de imágenes de soporte por clase.
        n_query (int): Número de imágenes de consulta por clase.
        model (tf.keras.Model): Modelo de red neuronal para extraer características.

    Salida:
        loss (tf.Tensor): Valor de la función de pérdida calculada.
        target_inds_onehot (tf.Tensor): Etiquetas verdaderas en formato one-hot encoding.
        log_p_y (tf.Tensor): Log-probabilidades predichas.
    """

    # Separa las imágenes de soporte y consulta del diccionario 'sample'
    x_support = sample['imagenes'][:, :n_support]
    x_query = sample['imagenes'][:, n_support:]

    # Reajusta las dimensiones de las imágenes de soporte y consulta
    x_support = np.reshape(x_support, (x_support.shape[0] * x_support.shape[1], *x_support.shape[2:]))
    x_query = np.reshape(x_query, (x_query.shape[0] * x_query.shape[1], *x_query.shape[2:]))

    # Convierte las imágenes a tensores y normaliza sus valores (0-255 a 0-1)
    x_support = tf.convert_to_tensor(x_support / 255.0, dtype=tf.float32)
    x_query = tf.convert_to_tensor(x_query / 255.0, dtype=tf.float32)

    # Genera las etiquetas verdaderas para las imágenes de consulta en formato one-hot
    target_inds = [i // n_query for i in range(n_way * n_query)]
    target_inds_onehot = tf.keras.utils.to_categorical(target_inds, num_classes=n_way)
    target_inds_onehot = tf.convert_to_tensor(target_inds_onehot)

    # Usa el modelo para extraer las características de las imágenes de soporte y consulta
    z_support = model(x_support)
    z_query = model(x_query)

    # Calcula el prototipo (representación promedio) para cada clase a partir de las características de soporte
    z_proto = tf.reduce_mean(tf.reshape(z_support, (n_way, n_support, -1)), axis=1)

    # Calcula la distancia euclídea entre las características de consulta y los prototipos
    dists = euclidean_dist(z_query, z_proto)

    # Calcula las log-probabilidades predichas a partir de las distancias
    log_p_y = tf.nn.log_softmax(-dists, axis=1)

    # Calcula la función de pérdida usando las etiquetas verdaderas y las log-probabilidades predichas
    loss = floss(target_inds_onehot, log_p_y)

    return loss, target_inds_onehot, log_p_y


In [ ]:
def train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epocas, tamaño_epoca):
    """
    Entrena el modelo utilizando una red prototipica.

    Args:
        model (tf.keras.Model): Modelo de la red convolucional a entrenar.
        optimizer (tf.keras.optimizers.Optimizer): Optimizador a utilizar.
        train_x (np.array): Imágenes del conjunto de entrenamiento.
        train_y (np.array): Etiquetas del conjunto de entrenamiento.
        n_way (int): Número de clases diferentes para la clasificación.
        n_support (int): Número de ejemplos de soporte por clase.
        n_query (int): Número de ejemplos de consulta por clase.
        max_epocas (int): Número máximo de épocas de entrenamiento.
        tamaño_epoca (int): Número de episodios por época.
    """

    # Define el plan de ajuste para el learning rate
    scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
        0.001, decay_steps=2000, decay_rate=0.5, staircase=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=scheduler)

    # Listas para almacenar los valores de pérdida y precisión parcial para cada época
    loss_parcial = []
    acc_parcial = []

    for epoca in range(max_epocas):
        # Inicialización de la pérdida y precisión para cada época
        loss_actual = 0.0
        acc_actual = 0.0

        for episodio in tnrange(tamaño_epoca):
            # Registra las operaciones para las cuales se calcularán los gradientes
            with tf.GradientTape() as tape:
                # Crea una muestra aleatoria optimizada
                sample = crea_muestra(n_way, n_support, n_query, train_x, train_y)
                # Calcula la pérdida y las log-probabilidades usando la función Proto
                loss, target_inds_onehot, log_p_y = Proto(sample, n_way, n_support, n_query, model)
                # Calcula los gradientes
                gradients = tape.gradient(loss, model.trainable_variables)
                # Aplica los gradientes para actualizar el modelo
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            # Calcula la precisión para este episodio
            accuracy = accuracyProt(target_inds_onehot, log_p_y, n_way, n_query)
            # Acumula la pérdida y precisión
            loss_actual += loss
            acc_actual += accuracy

            # Guarda la pérdida y precisión media cada 500 episodios
            if (episodio + 1) % 500 == 0:
                loss_parcial.append(loss_actual / 500)
                acc_parcial.append(acc_actual / 500)
                loss_actual = 0.0  # resetea la pérdida acumulada
                acc_actual = 0.0   # resetea la precisión acumulada

        # Calcula la pérdida y precisión media de la época y la imprime
        loss_epoca = sum(loss_parcial[-(tamaño_epoca // 500):]) / (tamaño_epoca // 500)
        acc_epoca = sum(acc_parcial[-(tamaño_epoca // 500):]) / (tamaño_epoca // 500)
        print(f'Epoch {epoca + 1} -- Loss: {loss_epoca:.4f} Acc: {acc_epoca:.4f}')

    return loss_parcial, acc_parcial



In [ ]:
def crea_modelo():
    return red_convolucional((84, 84, 3), 256)

In [ ]:
# Define las configuraciones de entrenamiento
n_ways = [60]           # Lista de diferentes valores de 'n_way' a probar
n_support = 5           # Número de ejemplos de soporte por clase
n_query = 5             # Número de ejemplos de consulta por clase

train_x = trainx        # Conjunto de entrenamiento de imágenes
train_y = trainy        # Etiquetas del conjunto de entrenamiento

max_epocas = 10         # Número máximo de épocas de entrenamiento
tamaño_epoca = 2000     # Número de episodios por época

# Loop a través de diferentes configuraciones 'n_way'
for n_way in n_ways:
    # Crea un nombre único para cada configuración de modelo
    model_name = f'modelProImag{n_way}w{n_support}s2'

    # Crea un nuevo modelo prototípico para esta configuración
    model = crea_modelo()

    # Define el optimizador para el entrenamiento
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Entrena el modelo con la configuración actual de 'n_way'
    loss, acc = train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epocas, tamaño_epoca)

    # Guarda el modelo entrenado en el disco
    model.save(f'{model_name}.h5')


In [ ]:
def test(model, test_x, test_y, n_way, n_support, n_query, episodios_test):
    """
    Prueba el modelo prototypical
    Args:
        model: modelo entrenado
        test_x (np.array): imágenes del conjunto de prueba
        test_y (np.array): etiquetas del conjunto de prueba
        n_way (int): número de clases en una tarea de clasificación
        n_support (int): número de ejemplos etiquetados por clase en el conjunto de soporte
        n_query (int): número de ejemplos etiquetados por clase en el conjunto de consulta
        episodios_test (int): número de episodios para probar
    """

    # Inicializar las métricas de pérdida y precisión
    loss_actual = 0.0
    acc_actual = 0.0

    # Iterar a través de cada episodio de prueba
    for episode in tnrange(episodios_test):
        # Crear una muestra optimizada a partir del conjunto de prueba
        sample = crea_muestra(n_way, n_support, n_query, test_x, test_y)

        # Calcular la pérdida y las probabilidades logarítmicas con la función Proto
        loss, target_inds_onehot, log_p_y  = Proto(sample, n_way, n_support, n_query, model)

        # Calcular la precisión del modelo en esta muestra
        accuracy = accuracyProt(target_inds_onehot, log_p_y, n_way, n_query)

        # Acumular la pérdida y la precisión para calcular el promedio posteriormente
        loss_actual += loss
        acc_actual += accuracy

    # Calcular las métricas promedio a lo largo de todos los episodios de prueba
    avg_loss = loss_actual / episodios_test
    avg_acc = acc_actual / episodios_test

    # Imprimir los resultados de la prueba
    print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

    # Devolver la precisión promedio
    return avg_acc


In [ ]:
# Define los parámetros de prueba:
# n_ways_test: Lista que contiene el número de clases que se utilizarán en las tareas de prueba.
# n_support: Número de ejemplos etiquetados por clase en el conjunto de soporte.
# n_query: Número de ejemplos etiquetados por clase en el conjunto de consulta.
# test_episode: Número total de episodios para realizar pruebas.

n_ways_test = [5] #La lista se puede ampliarse con el resto de n-ways que se quieran probar
n_support = 5 # 5-shot, se puede cambiar a 1-shot
n_query = 5 # 5 imagenes de prueba por cada clase.
episodios_test = 500

# Itera a través de cada valor de 'n_way' en la lista 'n_ways_test'
for n_way in n_ways_test:
    # Define el nombre del archivo del modelo previamente entrenado para cargarlo (el modelo debe tener el mismo nombre)
    nombre_modelo = f'modelProImag60w5s.h5'

    # # Carga el modelo con el nombre definido
    model = load_model(nombre_modelo)

    # Realiza pruebas en el modelo cargado usando los parámetros definidos y el conjunto de datos de prueba (testx y testy)
    accuracy = test(model, testx, testy, n_way, n_support, n_query, episodios_test)

    # Imprime la precisión obtenida en la prueba
    print(f'Model {nombre_modelo} tested with accuracy {accuracy}')
